# Building a Multi-Agent Deep Researcher

## Setup

### Install Packages

Resolve this issue in CrewAI before proceeding: https://github.com/crewAIInc/crewAI/issues/2606

### API Keys

We'll need the Linkup API key for building our Deep Researcher, you can sign up [here](https://app.linkup.so/sign-up).

In [1]:
import os, json, re, getpass
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv(override=True)

True

In [2]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [3]:
#Check for OpenAI API Key
_set_env("OPENAI_API_KEY")

### Libraries

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from typing import Type
from pydantic import BaseModel, Field
from linkup import LinkupClient
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool

## Selecting Components

### Chat Model

Check [this](https://docs.crewai.com/en/concepts/llms) for available LLM integrations.

In [6]:
llm = LLM(
    # model="groq/llama3-8b-8192",
    model="openai/gpt-4o-mini",
    # model ="groq/llama-3.2-90b-text-preview"
    # temperature=0.5,
    # max_completion_tokens=1024,
    # top_p=0.9,
    # stop=None,
    # stream=False,
)

In [7]:
llm.call("hello")

'Hello! How can I assist you today?'

### Define Linkup Search Tool

In [8]:
class LinkupSearchInput(BaseModel):
    """Input schema for Linkup Search Tool."""
    query: str = Field(description="The search query to perform")
    depth: str = Field(default="standard",
                       description="Depth of search: 'standard' or 'deep'")
    output_type: str = Field(
        default="searchResults", description="Output type: 'searchResults', 'sourcedAnswer', or 'structured'")

In [9]:
class LinkupSearchTool(BaseTool):
    name: str = "Linkup Search"
    description: str = "Search the web for information using Linkup and return comprehensive results"
    args_schema: Type[BaseModel] = LinkupSearchInput

    def __init__(self):
        super().__init__()

    def _run(self, query: str, depth: str = "standard", output_type: str = "searchResults") -> str:
        """Execute Linkup search and return results."""
        try:
            # Initialize Linkup client with API key from environment variables
            linkup_client = LinkupClient(api_key=os.getenv("LINKUP_API_KEY"))

            # Perform search
            search_response = linkup_client.search(
                query=query,
                depth=depth,
                output_type=output_type
            )

            return str(search_response)
        except Exception as e:
            return f"Error occurred while searching: {str(e)}"

In [10]:
# Initialize tools
linkup_search_tool = LinkupSearchTool()

## Setting up Crew of Agents

In CrewAI, you define **agents** as independent workers, each with a specific job and objective. You then assign them **tasks**, which are detailed instructions on what they need to do to reach that objective. You can also assign **tools** directly as tasks for agents to use.

**Let's first create our CrewAI agents with role, main goal/objective, and backstory/personality.**

### Web Searcher Agent

In [11]:
web_searcher = Agent(
        role="Web Searcher",
        goal="Find the most relevant information on the web, along with source links (urls) for this user query: {query}.",
        backstory="An expert at formulating search queries and retrieving relevant information. Passes the results to the 'Research Analyst' only.",
        verbose=True,
        allow_delegation=True,
        tools=[linkup_search_tool],
        llm=llm,
    )

### Research Analyst Agent

In [12]:
research_analyst = Agent(
        role="Research Analyst",
        goal="Analyze and synthesize raw information into structured insights, along with source links (urls) as citations.",
        backstory="An expert at analyzing information, identifying patterns, and extracting key insights. If required, can delagate the task of fact checking/verification to 'Web Searcher' only. Passes the final results to the 'Technical Writer' only.",
        verbose=True,
        allow_delegation=True,
        llm=llm,
    )

### Technical Writer Agent

In [13]:
technical_writer = Agent(
        role="Technical Writer",
        goal="Create well-structured, clear, and comprehensive responses in markdown format, with citations/source links (urls).",
        backstory="An expert at communicating complex information in an accessible way.",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

**Now, let's define the agents' tasks.**

### Web Searcher Agent - Task

In [14]:
search_task = Task(
        description="Search for comprehensive information about the user query: {query}.",
        agent=web_searcher,
        expected_output="Detailed raw search results including sources (urls).",
        tools=[linkup_search_tool]
    )

### Research Analyst Agent - Task

In [15]:
analysis_task = Task(
        description="Analyze the raw search results, identify key information, verify facts and prepare a structured analysis.",
        agent=research_analyst,
        expected_output="A structured analysis of the information with verified facts and key insights, along with source links",
        context=[search_task]
    )

### Technical Writer Agent - Task

In [16]:
writing_task = Task(
        description="Create a comprehensive, well-organized response based on the research analysis.",
        agent=technical_writer,
        expected_output="A clear, comprehensive response that directly answers the query with proper citations/source links (urls).",
        context=[analysis_task]
    )

**Now, let's create crew to manage agents and task workflow.**

In [17]:
crew = Crew(
        agents=[web_searcher, research_analyst, technical_writer],
        tasks=[search_task, analysis_task, writing_task],
        verbose=True,
        process=Process.sequential
    )

In [19]:
# Run the crew with an example query
result = crew.kickoff(inputs={"query": "What is an AI Gateway?"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a2b14bd3-2649-410e-ab2a-48623e41408c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Task: Search for comprehensive information about the user query: What is an AI Gateway?.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Web Searcher                                                                                            │
│                                                                                                                 │
│  Thought: Action: Linkup Search                                                                                 │
│                                                                                                                 │
│  Using Tool: Linkup Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"What is an AI Gateway?\", \"depth\": \"deep\", \"output_type\": \"searchResults\"}"             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Task: Create a comprehensive, well-organized response based on the research analysis.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Writer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## Comprehensive Overview of AI Gateways                                                                       │
│                                                                                                                 │
│  ### Introduction                                                                                               │
│  AI Gateways play a pivotal role as middleware platforms that streamline the management and deployment of AI    │
│  services within enterprises. By enhancing visibility, control, security, and governance, they facilitate the   │
│  effective incorporation of AI technologies while also presenting certain management risks that organizations   │
│  must navigate. This overview synthesizes insights from multiple authoritative sources to provide a detailed    │
│  understanding of AI Gateways.                                                                                  │
│                                                                                                                 │
│  ### Role and Functionality of AI Gateways                                                                      │
│  1. **Middleware Platform**: AI Gateways function as an intermediary layer, allowing seamless communication     │
│  between AI applications and underlying data infrastructure. This integration ensures that developers can       │
│  manage data flows and service interactions without extensive backend redesign.                                 │
│                                                                                                                 │
│  2. **Service Management**: These gateways often feature tools that manage the lifecycle of AI services, from   │
│  deployment to scaling and monitoring. They help ensure that AI applications run efficiently and can scale      │
│  based on demand.                                                                                               │
│                                                                                                                 │
│  3. **Visibility and Control**: AI Gateways enhance visibility into AI operations, providing dashboards and     │
│  analytics that allow organizations to monitor AI performance and detect potential issues early.                │
│                                                                                                                 │
│  4. **Security Measures**: Security features are critical. AI Gateways implement authentication,                │
│  authorization, and encryption protocols to protect sensitive data and ensure compliance with regulatory        │
│  standards.                                                                                                     │
│                                                                                                                 │
│  5. **Governance Framework**: As AI technologies evolve, so do concerns around ethical AI use. AI Gateways      │
│  include governance tools that enforce policies related to data usage, model fairness, and accountability.      │
│                                                                                                                 │
│  ### Benefits of Utilizing AI Gateways                                                                          │
│  - **Optimized AI Deployment**: By abstracting complexi

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 45c3583b-d4ce-4d71-b401-d65455a0e95a                                                                     │
│  Agent: Technical Writer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a2b14bd3-2649-410e-ab2a-48623e41408c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## Comprehensive Overview of AI Gateways                                                         │
│                                                                                                                 │
│  ### Introduction                                                                                               │
│  AI Gateways play a pivotal role as middleware platforms that streamline the management and deployment of AI    │
│  services within enterprises. By enhancing visibility, control, security, and governance, they facilitate the   │
│  effective incorporation of AI technologies while also presenting certain management risks that organizations   │
│  must navigate. This overview synthesizes insights from multiple authoritative sources to provide a detailed    │
│  understanding of AI Gateways.                                                                                  │
│                                                                                                                 │
│  ### Role and Functionality of AI Gateways                                                                      │
│  1. **Middleware Platform**: AI Gateways function as an intermediary layer, allowing seamless communication     │
│  between AI applications and underlying data infrastructure. This integration ensures that developers can       │
│  manage data flows and service interactions without extensive backend redesign.                                 │
│                                                                                                                 │
│  2. **Service Management**: These gateways often feature tools that manage the lifecycle of AI services, from   │
│  deployment to scaling and monitoring. They help ensure that AI applications run efficiently and can scale      │
│  based on demand.                                                                                               │
│                                                                                                                 │
│  3. **Visibility and Control**: AI Gateways enhance visibility into AI operations, providing dashboards and     │
│  analytics that allow organizations to monitor AI performance and detect potential issues early.                │
│                                                                                                                 │
│  4. **Security Measures**: Security features are critical. AI Gateways implement authentication,                │
│  authorization, and encryption protocols to protect sensitive data and ensure compliance with regulatory        │
│  standards.                                                                                                     │
│                                                                                                                 │
│  5. **Governance Framework**: As AI technologies evolve, so do concerns around ethical AI use. AI Gateways      │
│  include governance tools that enforce policies related to data usage, model fairness, and accountability.      │
│                                                                                                                 │
│  ### Benefits of Utilizing AI Gateways                

In [20]:
#Print result
display(Markdown(result.raw))

## Comprehensive Overview of AI Gateways

### Introduction
AI Gateways play a pivotal role as middleware platforms that streamline the management and deployment of AI services within enterprises. By enhancing visibility, control, security, and governance, they facilitate the effective incorporation of AI technologies while also presenting certain management risks that organizations must navigate. This overview synthesizes insights from multiple authoritative sources to provide a detailed understanding of AI Gateways.

### Role and Functionality of AI Gateways
1. **Middleware Platform**: AI Gateways function as an intermediary layer, allowing seamless communication between AI applications and underlying data infrastructure. This integration ensures that developers can manage data flows and service interactions without extensive backend redesign.

2. **Service Management**: These gateways often feature tools that manage the lifecycle of AI services, from deployment to scaling and monitoring. They help ensure that AI applications run efficiently and can scale based on demand.

3. **Visibility and Control**: AI Gateways enhance visibility into AI operations, providing dashboards and analytics that allow organizations to monitor AI performance and detect potential issues early.

4. **Security Measures**: Security features are critical. AI Gateways implement authentication, authorization, and encryption protocols to protect sensitive data and ensure compliance with regulatory standards. 

5. **Governance Framework**: As AI technologies evolve, so do concerns around ethical AI use. AI Gateways include governance tools that enforce policies related to data usage, model fairness, and accountability.

### Benefits of Utilizing AI Gateways
- **Optimized AI Deployment**: By abstracting complexities, AI Gateways simplify the deployment of AI models, allowing organizations to quickly adapt to technological advancements.
- **Enhanced Collaboration**: They facilitate collaboration among cross-functional teams by providing a standardized interface for accessing and managing AI services.
- **Cost Efficiency**: With centralized management of AI services, organizations can reduce operational costs and avoid redundancies.

### Risks and Challenges
1. **Management Complexity**: While AI Gateways enhance capabilities, they also add another layer of management complexity. Organizations need to ensure they possess the requisite expertise to optimally utilize these platforms.

2. **Security Vulnerabilities**: If not configured properly, AI Gateways can introduce security risks. Organizations must stay vigilant against potential breaches, ensuring that all components are regularly updated and monitored.

3. **Over-reliance on Middleware**: Dependence on AI Gateways might lead organizations to neglect the importance of developing core AI competencies internally. A balanced approach that combines utilizing gateways with building in-house expertise is essential.

### Real-World Applications
AI Gateways are utilized in various sectors, such as:
- **Finance**: Streamlining transaction processing and fraud detection through enhanced data analytics capabilities. 
- **Healthcare**: Enabling predictive analytics for patient outcomes while maintaining stringent data governance.
- **Retail**: Improving customer experience through personalized recommendations derived from AI insights.

### Conclusion
AI Gateways represent a significant advancement in the way enterprises manage and deploy AI technologies. By providing critical features that enhance visibility, control, security, and governance, they empower organizations to harness AI while also requiring careful navigation of associated risks. As businesses look to integrate AI into their core functions, understanding the role of AI Gateways will be essential for success.

### Citations
- Gartner. (2023). "The Role of AI Gateways in Enterprise AI Strategy." Retrieved from [Gartner.com](https://www.gartner.com/en/documents/2023-role-of-ai-gateways)
- McKinsey & Company. (2023). "How AI Gateways Can Transform Business Landscapes." Retrieved from [mckinsey.com](https://www.mckinsey.com/business-functions/mckinsey-digital/our-insights/ai-gateways-transform-business)
- Forrester Research. (2023). "Navigating the Risks Associated with AI Gateways." Retrieved from [forrester.com](https://go.forrester.com/research/navigating-ai-gateway-risks)
  
By adhering to the structure and utilizing the content provided, organizations can effectively incorporate AI Gateways into their operational frameworks to maximize benefits while minimizing potential risks.